In [1]:
from components.balloon import Balloon, BalloonStatus
from components.network import Network
from components.networkanalyzer import NetworkAnalyzer as na
from components.plothelper import PlotHelper as ph
from components.windmap import WindMap as wm
from components.sample_antennas import *
from sims.antennas import *

import numpy as np
import random

In [3]:
reconfigs = []
meaningful = []

passes = 0
while passes < 10:
    MyWindMap = wm(500, 500, 1)
    MyWindMap.create_windmap()
    MyWindMap.bound(4)
    
    MyPlotHelper = ph(MyWindMap)
    MyPlotHelper.populate(show_scale=False)
    # MyPlotHelper.plot(debug=False)
    
    NumBalloons = 5
    Colors = ["red", "blue", "green", "orange", "purple"]
    Balloons = []
    GenericPath = []
    TickResponses = []
    AntennaToAdd = MolexNoPL
    
    for i in range(1, NumBalloons+1):
        Balloons.append(Balloon(MyWindMap, str(i), Colors[i-1]))
        GenericPath.append(str(i))
        start_pos_modifier = random.randint(-2, 2)
        Balloons[i-1].set_start_pos(-20 + start_pos_modifier[i], -45 + start_pos_modifier[i])
        TickResponses.append(1)
        Balloons[i-1].add_antenna(AntennaToAdd)
    
    # MyPlotHelper.plot_balloons(B1, B2, B3, B4, B5)
    # MyPlotHelper.zoom_in(100, 100)
    
    NA = na(*Balloons)
    # MyNetworkAnalyzer.show_pointing(100, MyFirstBalloon, MySecondBalloon, MyThirdBalloon, MyFourthBalloon)
    
    MyNetwork = Network(*Balloons,
                        start=Balloons[0],
                        end=Balloons[NumBalloons-1],
                        generic_path=GenericPath)
        
    for i in range(1, NumBalloons+1):
        while TickResponses[i-1]:
            TickResponses[i-1] = Balloons[i-1].tick()
       
    sim_dud = False     
    for i in range(1, NumBalloons+1):
        if Balloons[i-1].current_status != BalloonStatus.FALL:
            sim_dud = True
            
    if sim_dud:
        print("\n == SIM DUD == \n")
        continue
            
    
    for i in range(1, 8000):
        MyNetwork.recalculate(i, NA.rp)
        MyNetwork.transmit(i, NA.rp)
    
    num_meaningfuls = len(list(filter(lambda x: x['meaningful'] == True, MyNetwork.reconfigures)))
    
    print("\n== SIMULATION", passes, "COMPLETE ==")
    print("Reconfigurations:", len(MyNetwork.reconfigures))
    reconfigs.append(len(MyNetwork.reconfigures))
    print("Meaningful (above threshhold) reconfigs:", num_meaningfuls)
    print("Percentage meaningful:", (num_meaningfuls/len(MyNetwork.reconfigures))*100, "\n")
    meaningful.append(num_meaningfuls)
    
    passes += 1

print("== ALL SIMS COMPLETE ==")
print("Mean reconfigurations:", np.mean(reconfigs))
print("Mean meaningful reconfigurations:", np.mean(meaningful))
print("St.dev of reconfigurations:", np.std(reconfigs))
print("Variance of meaningful reconfigurations:", np.std(meaningful))

1  popped at  7068.5
1  hit the ground at time t= 9334.5
2  popped at  5962.0
2  hit the ground at time t= 8227.0
3  popped at  7560.5
3  hit the ground at time t= 9825.0
4  popped at  7800.0
4  hit the ground at time t= 10063.5
5  popped at  6221.0
5  hit the ground at time t= 8483.0

== SIMULATION 0 COMPLETE ==
Reconfigurations: 204
Meaningful (above threshhold) reconfigs: 69
Percentage meaningful: 33.82352941176471 

1  popped at  6823.5
1  hit the ground at time t= 9088.5
2  popped at  6231.0
2  hit the ground at time t= 8493.5
3  popped at  7510.0
3  hit the ground at time t= 9772.5
4  popped at  5963.5
4  hit the ground at time t= 8229.5
5  popped at  6423.0
5  hit the ground at time t= 8687.5

== SIMULATION 1 COMPLETE ==
Reconfigurations: 361
Meaningful (above threshhold) reconfigs: 100
Percentage meaningful: 27.70083102493075 

1  popped at  7327.0
1  hit the ground at time t= 9588.5
2  popped at  6361.5
2  hit the ground at time t= 8626.5
3  popped at  7695.0
3  hit the ground